## Домашняя работа
Группа: РИМ-150950

ФИО: Эрмиш Александр Александрович

## Задание 1

Используя библиотеку requests , выполните GET-запрос к публичному API и извлеките
данные. Помимо запроса из примера выполните запрос на любой иной url и напишите к нему проверку.

Пример: запросите `https://api.agify.io?name=Ivan` (сервис предсказания возраста
по имени). Напишите функцию, которая возвращает предсказанный возраст.

In [1]:
import requests
from typing import Optional, List, Dict

In [3]:
def get_predicted_age(name) -> Optional[int]:
    # Базовый URL API Agify
    base_url = "https://api.agify.io"

    # Параметры запроса передаются в виде словаря → requests автоматически
    # сформирует строку `?name=Ivan` и корректно закодирует значение.
    params = {"name": name}

    try:
        # Выполняем запрос. timeout защищает от «залипания» при падении сервера.
        response = requests.get(base_url, params=params, timeout=10)
        # Проверяем HTTP‑статус. raise_for_status() бросит исключение,
        # если статус 4xx или 5xx.
        response.raise_for_status()
    
    except requests.RequestException as exc:
        # Если что‑то пошло не так (нет соединения, таймаут, 4xx/5xx),
        # выводим сообщение в консоль (можно заменить на логирование) и возвращаем None.
        print(f"[ERROR] Не удалось выполнить запрос к Agify: {exc}")
        return None
    
    # Декодируем JSON‑тело ответа.
    data = response.json()

    # В ответе может быть поле `age`. Если оно отсутствует (null), возвращаем None.
    age = data.get("age")
    if age is None:
        # API может вернуть null, если имя слишком редкое.
        return None
    
    # Убедимся, что полученный возраст действительно целое число.
    # (В официальной документации он уже int, но проверка делает код более надёжным.)
    if isinstance(age, int):
        return age
    else:
        # Если тип неожиданный, тоже считаем, что предсказание недоступно.
        print(f"[WARN] Получен некорректный тип возраста: {type(age)}")
        return None
    
def get_nationalities(name: str) -> List[Dict[str, float]]:
    """
    Делает запрос к API `https://api.nationalize.io`, которое предсказывает
    вероятные страны по имени, и проверяет корректность полученных данных.
    Параметры
    ----------
    name: str
        Имя, для которого делаем предсказание.
    Возврат
    -------
    list[dict]
        Список словарей вида {'country_id': 'RU', 'probability': 0.42}.
        Если запрос не удался – возвращаем пустой список.
    """
    url = "https://api.nationalize.io"
    params = {"name": name}
    try:
        resp = requests.get(url, params=params, timeout=10)
        resp.raise_for_status()
    except requests.RequestException as exc:
        print(f"[ERROR] Ошибка запроса к Nationalize: {exc}")
        return []
    payload = resp.json()
    # Поле `country` содержит список предсказаний.
    countries = payload.get("country", [])
    if not isinstance(countries, list):
        # Если API вернул неожиданную структуру – считаем, что данных нет.
        print("[WARN] Поле `country` имеет неверный тип.")
        return []
    # Приведём каждый элемент к нужному виду и проверим вероятность.
    cleaned: List[Dict[str, float]] = []
    total_prob = 0.0
    for entry in countries:
        country_id = entry.get("country_id")
        prob = entry.get("probability")
        # Базовые проверки типов
        if isinstance(country_id, str) and isinstance(prob, (float, int)):
            prob = float(prob)      # гарантируем тип float
            total_prob += prob
            cleaned.append({"country_id": country_id, "probability": prob})
        else:
            print(f"[WARN] Некорректный элемент в ответе: {entry}")
    # Сумма вероятностей не должна превышать 1 (погрешность допускается из‑за округления).
    if total_prob > 1.0001:
        print(f"[WARN] Сумма вероятностей ({total_prob}) превышает 1 – данные могут быть некорректны.")
        # Можно решить, обрезать, нормализовать или просто вернуть пустой список.
        # Здесь просто возвращаем уже полученный список, оставляя ответственность вызывающему коду.
    return cleaned

## Проверка задания 1 для примера

In [7]:
age = get_predicted_age("Ivan")  # Ваша функция
print("Age is int or None:", isinstance(age, int) or age is None)

Age is int or None: True


In [12]:
test_name = 'Ann'
nationalities = get_nationalities(test_name)
print("\nПредсказанные национальности:")
if nationalities:
    for i, rec in enumerate(nationalities, 1):
        print(f"  {i}. Страна: {rec['country_id']}, вероятность: {rec['probability']:.2%}")
    # Дополнительная проверка: сумма вероятностей
    total = sum(r["probability"] for r in nationalities)
    print(f"  Суммарная вероятность: {total:.2%}")
else:
    print("  Данные не получены.")


Предсказанные национальности:
  1. Страна: US, вероятность: 8.04%
  2. Страна: LC, вероятность: 4.95%
  3. Страна: CN, вероятность: 4.52%
  4. Страна: SG, вероятность: 4.25%
  5. Страна: MY, вероятность: 4.19%
  Суммарная вероятность: 25.94%


## Задание 2

Дана строка с контактными данными:

`Для связи: test@example.com, support@test.org. Не используйте адреса: spam@badsite.com.`

С помощью модуля `re` извлеките все email-адреса.

In [13]:
import re

In [27]:
def extract_emails(text: str) -> list:
    """
    Находит все e‑mail‑адреса в переданном тексте.
    Параметры
    ----------
    text : str
        Строка, в которой нужно искать адреса.
    Возврат
    -------
    list[str]
        Список найденных e‑mail‑адресов (в том порядке, в котором они
        встречаются в тексте). Если ничего не найдено – пустой список.
    """
    # Регулярные выражения:
    #   \b          – граница слова (чтобы не захватывать лишние символы)
    #   [\w.+-]+    – «имя» части адреса: буквы, цифры, подчеркивания,
    #                точка, плюс, тире (самые распространённые символы)
    #   @           – символ «@»
    #   [\w.-]+     – домен: буквы, цифры, точка, тире
    #   \.\w{2,}    – точка + минимум две буквы (доменная зона, например .com)
    #   \b          – граница слова в конце
    email_pattern = r'\b[\w.+-]+@[\w.-]+\.\w{2,}\b'
    # re.findall возвращает список всех совпадений паттерна в строке.
    matches = re.findall(email_pattern, text)
    return matches


## Проверка задания 2

In [31]:
text = "Для связи: test@example.com, support@test.org. Не используйте адреса: spam@badsite.com."

In [32]:
emails = extract_emails(text)
expected_emails = ["test@example.com", "support@test.org", "spam@badsite.com"]

print("Emails extracted correctly:", emails == expected_emails)

Emails extracted correctly: True


### Полезные ссылки

[Теория по регулярным выражениям](https://selectel.ru/blog/courses/regex-course/)